In [50]:
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)


import tensorflow as tf
sess = tf.InteractiveSession()

Extracting MNIST_data/train-images-idx3-ubyte.gz


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [52]:
# this code is all copied from "Deep MNIST for experts" tutorial
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1,28,28,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)


W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [54]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
for i in range(1000):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.04


step 100, training accuracy 0.78


step 200, training accuracy 0.92


step 300, training accuracy 0.94


step 400, training accuracy 0.96


...

step 600, training accuracy 0.9


step 700, training accuracy 0.92


step 800, training accuracy 0.94


step 900, training accuracy 1


test accuracy 0.9634


In [70]:
N = 10
# get N images of '2' from the dataset
i = 0
res = []
for (idx, num) in enumerate(mnist.test.labels):
    if num[2] == 1:  # labels are 1-hot encoded
        # check if correctly predicted
        correct = correct_prediction.eval(feed_dict = {
            x: mnist.test.images[idx][None, :],
            y_: mnist.test.labels[idx][None, :],
            keep_prob: 1.0
        })
        if not correct:
            break
        res.append(idx)
        i += 1
        if i >= N:
            break

rows = np.array(res)
twos_x = mnist.test.images[rows, :]
twos_y = mnist.test.labels[rows, :]

In [141]:
# Restrict the cross entropy calculation to only 2 and 6 labels.
# Then increasing cross entropy is equivalent to moving towards label 6.

y_conv_2_6 = tf.transpose(tf.gather(tf.transpose(y_conv), indices=[2,6]))
y_2_6 = tf.transpose(tf.gather(tf.transpose(y_), indices=[2,6]))

cross_entropy_2_6 = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=y_conv_2_6, labels=y_2_6)
)

x_grad = tf.gradients(cross_entropy_2_6, x)[0]
y_pred = tf.argmax(y_conv,1)

fool_x = twos_x
for i in range(3):
    x_grad_vals = x_grad.eval(feed_dict = {
       x: fool_x,
      y_: twos_y,
     keep_prob: 1.0
    })

    eps = 1
    fool_x = fool_x + eps * np.sign(x_grad_vals)
    new_pred = y_pred.eval(feed_dict = {
        x: fool_x,
        keep_prob: 1.0
    })
    print(new_pred)

/home/michbad/anaconda3/envs/tensorflow/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[6 6 5 1 4 6 1 6 5 6]
[6 6 6 6 6 6 6 6 6 6]
[6 6 6 6 6 6 6 6 6 6]


In [139]:
x.value

AttributeError: 'Tensor' object has no attribute 'value'